In [61]:
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx

### Singe Random Network
### network config
AVG_DEGREE = 6
NUM_NODE = 200
GIANT_COMPONENT_THRESHOLD=0.90
NUM_ITER=100
PROTECT_HUB_DEGREE = 999999999
FILE="output.txt"
PROTECT_BETWEENNESS=0

ATTR_LAYER = "layer"
LAYER_A_TAG = "L_A"
LAYER_B_TAG = "L_B"
###

In [62]:
layer_a=[]
layer_b=[]
interconnect_stg=[]
attack_stg=[]
### Test
# layer_a+=["ER"]
# layer_b+=["ER"]
# interconnect_stg+=["Max_min_d"]
# attack_stg+=["Max_d"]

# ER-ER BA-BA random attack
# layer_a+=["ER"]*3
# layer_b+=["ER"]*3
# interconnect_stg+=["Random", "Max_min_d", "Max_max_d"]
# attack_stg+=["Random"]*3
# layer_a+=["BA"]*3
# layer_b+=["BA"]*3
# interconnect_stg+=["Random","Max_min_d", "Max_max_d"]
# attack_stg+=["Random"]*3
#
# #ER-ER BA-BA high_d target_atk
# layer_a+=["ER"]*2
# layer_b+=["ER"]*2
# interconnect_stg+=["Max_max_d"]*2
# attack_stg+=["Max_d","Min_d"]
#
# layer_a+=["BA"]*2
# layer_b+=["BA"]*2
# interconnect_stg+=["Max_max_d"]*2
# attack_stg+=["Max_d","Min_d"]
#
# # ER-BA BA-ER HH/R HDA/RA
# layer_a+=["ER","ER","ER","ER","BA","BA","BA","BA"]
# layer_b+=["BA","BA","BA","BA","ER","ER","ER","ER"]
# interconnect_stg+=["Random","Random","Max_max_d","Max_max_d","Random","Random","Max_max_d","Max_max_d"]
# attack_stg+=      ["Max_d",  "Random","Max_d",   "Random",   "Max_d", "Random",  "Max_d",   "Random"]
#
# #ER-ER BA-BA HBA
# layer_a+=["ER","ER","ER","BA","BA","BA"]
# layer_b+=["ER","ER","ER","BA","BA","BA"]
# interconnect_stg+=["Random"]*6
# attack_stg+=["Max_d","Random","Max_b"]*2
#
# #ER-ER R LDA/RA
# layer_a+=["ER","ER"]
# layer_b+=["ER","ER"]
# interconnect_stg+=["Random"]*2
# attack_stg+=["Min_d","Random"]
#


# layer_a+=["ER","ER"]
# layer_b+=["ER","ER"]
# PROTECT_HUB_DEGREE = 12
# interconnect_stg+=["Max_max_d"]*2
# attack_stg+=["Max_d","Random"]
# FILE_TAG="Prot_"
# layer_a+=["BA","BA"]
# layer_b+=["BA","BA"]
# PROTECT_HUB_DEGREE = 31
# interconnect_stg+=["Max_max_d"]*2
# attack_stg+=["Max_d","Random"]
# FILE_TAG="Prot_"
##

#### need to treat the output separetely
layer_a+=["BA","BA", "ER","ER"]
layer_b+=["BA","BA", "ER", "ER"]
interconnect_stg+=["Max_max_d"]*4
attack_stg+=["Max_d","Random"]*2
FILE_TAG="Prot_HB_"
PROTECT_BETWEENNESS=0.02
###

In [ ]:
fig_idx = 0
def new_fig():
    global fig_idx
    plt.figure(fig_idx)
    fig_idx += 1

def sort_max_degree(G):
    degree = dict(nx.degree(G))
    return max(degree, key=degree.get)

def sort_degree_desc(G):
    degree = dict(nx.degree(G))
    return {k: v for k, v in sorted(degree.items(), key=lambda item: item[1], reverse=True)}

def sort_degree_asc(G):
    degree = dict(nx.degree(G))
    return {k: v for k, v in sorted(degree.items(), key=lambda item: item[1], reverse=False)}

def sort_clustering_coefficient_desc(G):
    clustering_coefficient = dict(nx.clustering(G))
    return {k: v for k, v in sorted(clustering_coefficient.items(), key=lambda item: item[1], reverse=True)}

def sort_clustering_coefficient_asc(G):
    clustering_coefficient = dict(nx.clustering(G))
    return {k: v for k, v in sorted(clustering_coefficient.items(), key=lambda item: item[1], reverse=False)}

def sort_betweenness_centrality_desc(G):
    betweenness_centrality = dict(nx.betweenness_centrality(G))
    return {k: v for k, v in sorted(betweenness_centrality.items(), key=lambda item: item[1], reverse=True)}

def sort_betweenness_centrality_asc(G):
    betweenness_centrality = dict(nx.betweenness_centrality(G))
    return {k: v for k, v in sorted(betweenness_centrality.items(), key=lambda item: item[1], reverse=False)}

def draw_interdependent_G(G):
    global ATTR_LAYER, LAYER_A_TAG, LAYER_B_TAG
    layer_attr=nx.get_node_attributes(G,ATTR_LAYER)
    color_map = ['b' if layer_attr[node][0]== LAYER_A_TAG else 'g' for node in list(G.nodes())]
    new_fig()
    nx.draw(G, node_color=color_map, node_size=30)

### with certain average degree
##### ER
def generate_ER(avg_degree):
    global NUM_NODE
    p = np.true_divide(avg_degree, NUM_NODE)
    return nx.erdos_renyi_graph(NUM_NODE, p)
#### BA
def generate_BA(avg_degree):
    global NUM_NODE
    return nx.random_graphs.barabasi_albert_graph(NUM_NODE, round(avg_degree/2))

### inter-connect
### add properties of whether nodes is Layer A or B for further use
### return: G
####
#### e.g. max_max means nodes in layer A with highest degree centralities
####         connect to  nodes in layer A with highest degree centralities
def interconnect(G_layer_A, L_A, G_layer_B, L_B):
    G = nx.union(G_layer_A, G_layer_B)
    layer_mapping = dict()
    for idx in range(len(L_A.items())):
        node_A = list(L_A.keys())[idx]
        node_B = list(L_B.keys())[idx]
        layer_mapping.update(dict({node_A:node_B}))
        G.add_edge(node_A, node_B)
    return G, layer_mapping

def interconnect_random(G_layer_A, G_layer_B):
    shuffle = np.random.permutation(NUM_NODE)
    G_layer_A_nodes = list(G_layer_A.nodes())
    G_layer_A_nodes = np.asarray(G_layer_A_nodes)[shuffle]
    G_layer_A_nodes = {n:1 for n in G_layer_A_nodes }
    G_layer_B_nodes = dict(G_layer_B.nodes())
    return interconnect(G_layer_A, G_layer_A_nodes, G_layer_B, G_layer_B_nodes)


def interconnect_max_max_degree_centralities(G_layer_A, G_layer_B):
    L_A_degree_centrality = sort_degree_desc(G_layer_A)
    L_B_degree_centrality = sort_degree_desc(G_layer_B)
    return interconnect(G_layer_A, L_A_degree_centrality, G_layer_B, L_B_degree_centrality)
def interconnect_max_min_degree_centralities(G_layer_A, G_layer_B):
    L_A_degree_centrality = sort_degree_desc(G_layer_A)
    L_B_degree_centrality = sort_degree_asc(G_layer_B)
    return interconnect(G_layer_A, L_A_degree_centrality, G_layer_B, L_B_degree_centrality)
def interconnect_min_max_degree_centralities(G_layer_A, G_layer_B):
    L_A_degree_centrality = sort_degree_asc(G_layer_A)
    L_B_degree_centrality = sort_degree_desc(G_layer_B)
    return interconnect(G_layer_A, L_A_degree_centrality, G_layer_B, L_B_degree_centrality)
def interconnect_min_min_degree_centralities(G_layer_A, G_layer_B):
    L_A_degree_centrality = sort_degree_asc(G_layer_A)
    L_B_degree_centrality = sort_degree_asc(G_layer_B)
    return interconnect(G_layer_A, L_A_degree_centrality, G_layer_B, L_B_degree_centrality)

def interconnect_max_max_clustering(G_layer_A, G_layer_B):
    L_A_clustering_coefficient = sort_clustering_coefficient_desc(G_layer_A)
    L_B_clustering_coefficient = sort_clustering_coefficient_desc(G_layer_B)
    return interconnect(G_layer_A, L_A_clustering_coefficient, G_layer_B, L_B_clustering_coefficient)
def interconnect_min_max_clustering(G_layer_A, G_layer_B):
    L_A_clustering_coefficient = sort_clustering_coefficient_asc(G_layer_A)
    L_B_clustering_coefficient = sort_clustering_coefficient_desc(G_layer_B)
    return interconnect(G_layer_A, L_A_clustering_coefficient, G_layer_B, L_B_clustering_coefficient)
def interconnect_max_min_clustering(G_layer_A, G_layer_B):
    L_A_clustering_coefficient = sort_clustering_coefficient_desc(G_layer_A)
    L_B_clustering_coefficient = sort_clustering_coefficient_asc(G_layer_B)
    return interconnect(G_layer_A, L_A_clustering_coefficient, G_layer_B, L_B_clustering_coefficient)
def interconnect_min_min_clustering(G_layer_A, G_layer_B):
    L_A_clustering_coefficient = sort_clustering_coefficient_asc(G_layer_A)
    L_B_clustering_coefficient = sort_clustering_coefficient_asc(G_layer_B)
    return interconnect(G_layer_A, L_A_clustering_coefficient, G_layer_B, L_B_clustering_coefficient)


def interconnect_max_max_betweeness(G_layer_A, G_layer_B):
    L_A_betweenness_centrality = sort_betweenness_centrality_desc(G_layer_A)
    L_B_betweenness_centrality = sort_betweenness_centrality_desc(G_layer_B)
    return interconnect(G_layer_A, L_A_betweenness_centrality, G_layer_B, L_B_betweenness_centrality)
def interconnect_max_min_betweeness(G_layer_A, G_layer_B):
    L_A_betweenness_centrality = sort_betweenness_centrality_desc(G_layer_A)
    L_B_betweenness_centrality = sort_betweenness_centrality_asc(G_layer_B)
    return interconnect(G_layer_A, L_A_betweenness_centrality, G_layer_B, L_B_betweenness_centrality)
def interconnect_min_max_betweeness(G_layer_A, G_layer_B):
    L_A_betweenness_centrality = sort_betweenness_centrality_asc(G_layer_A)
    L_B_betweenness_centrality = sort_betweenness_centrality_desc(G_layer_B)
    return interconnect(G_layer_A, L_A_betweenness_centrality, G_layer_B, L_B_betweenness_centrality)
def interconnect_min_min_betweeness(G_layer_A, G_layer_B):
    L_A_betweenness_centrality = sort_betweenness_centrality_asc(G_layer_A)
    L_B_betweenness_centrality = sort_betweenness_centrality_asc(G_layer_B)
    return interconnect(G_layer_A, L_A_betweenness_centrality, G_layer_B, L_B_betweenness_centrality)

#### simulation
def simulate(interdependent_G:nx.Graph, L_A:nx.Graph, L_B:nx.Graph, layer_mapping, attack_nodes:[]):
    global M_MONTE_CARLO, NODE_DESTROY_THRESHOLD, LAYER_A_TAG, LAYER_B_TAG, PROTECT_HUB_DEGREE, PROTECT_BETWEENNESS
    G = interdependent_G.copy()
    ### it can be reformate better
    protected_nodes_A = []
    for n in L_A.nodes():
        if L_A.degree()[n] >= PROTECT_HUB_DEGREE :
            protected_nodes_A.append(n)
    protected_nodes_B = []
    for n in L_B.nodes():
        if L_B.degree()[n] > PROTECT_HUB_DEGREE :
            protected_nodes_B.append(n)
    if (PROTECT_BETWEENNESS >0 ):
        protect_nodes_num = round(len(interdependent_G.nodes())*PROTECT_BETWEENNESS)
        h_betweeness = sort_betweenness_centrality_desc(interdependent_G)
        protected_nodes = [t[0] for t in list(h_betweeness.items())[:protect_nodes_num]]
        for pn in protected_nodes:
            if  LAYER_A_TAG in pn:
                protected_nodes_A.append(pn)
            if LAYER_B_TAG in pn:
                protected_nodes_B.append(pn)

    attack_nodes_after_protect=[]
    for atk_n in attack_nodes:
        if (atk_n not in protected_nodes_A) and (atk_n not in protected_nodes_B):
            attack_nodes_after_protect.append(atk_n)

    G.remove_nodes_from(attack_nodes_after_protect)
    ## remove nodes in A and B
    L_A.remove_nodes_from(attack_nodes_after_protect)
    L_B.remove_nodes_from(attack_nodes_after_protect)
    for node in attack_nodes_after_protect:
        L_B.remove_node(layer_mapping[node])
        G.remove_node(layer_mapping[node])
    layer_mapping_B_to_A = {v: k for k, v in layer_mapping.items()}
    while True:
        L_A_ends=True
        L_B_ends = True
        ## find cluster in A
        clusters_A = list(nx.connected_components(L_A))
        ### for cluster A_i
        for c_A in clusters_A:
            c_A_to_B = [layer_mapping[n_A] for n_A in c_A]
            for n_B in c_A_to_B:
                edges =  L_B.edges(n_B)
                remove_edges = []
                for edge in edges:
                    if (edge[0] not in c_A_to_B) or (edge[1] not in c_A_to_B):
                        remove_edges.append(edge)
                if len(remove_edges)>0:
                    G.remove_edges_from(remove_edges)
                    L_B.remove_edges_from(remove_edges)
                    L_A_ends=False
        ## find cluster in B
        clusters_B = list(nx.connected_components(L_B))
        for c_B in clusters_B:
            c_B_to_A = [layer_mapping_B_to_A[n_B] for n_B in c_B]
            for n_A in c_B_to_A:
                edges =  L_A.edges(n_A)
                remove_edges = []
                for edge in edges:
                    if (edge[0] not in c_B_to_A) or (edge[1] not in c_B_to_A):
                        remove_edges.append(edge)
                if len(remove_edges)>0:
                    G.remove_edges_from(remove_edges)
                    L_A.remove_edges_from(remove_edges)
                    L_B_ends=False
        if (L_A_ends and L_B_ends) :
            break
    return measurement(G)


#### measurement
def measurement(G):
    global NUM_NODE, GIANT_COMPONENT_THRESHOLD
    ## measurement
    giant_component_exist = 1
    #### largest component size / G size
    miu = 0.0
    G_nodes_num = len(G.nodes())
    sub_components = sorted(nx.connected_components(G), key=len, reverse=True)
    G_0_size = len(sub_components[0])
    if (G_0_size/G_nodes_num <  GIANT_COMPONENT_THRESHOLD):
        giant_component_exist = 0

    miu = np.true_divide(G_0_size, G_nodes_num)

    ###
    ret = giant_component_exist, miu
    return ret


#### attack nodes
def attack_random_Layer_A(G_A, p):
    global NUM_NODE
    attack_num = round(NUM_NODE * p)
    shuffler = np.random.permutation(NUM_NODE)
    return np.asarray(G_A.nodes())[shuffler[:attack_num]]
def attack_high_degree_Layer_A(G_A, P):
    global  LAYER_A_TAG

    A_copy = nx.Graph(G_A)
    num = round(P*len(G_A.nodes()))
    atk = []
    for i in range(num):
        atk_n = sort_max_degree(A_copy)
        atk.append(atk_n)
        A_copy.remove_node(atk_n)
    return  atk

def attack_high_betweenness_Layer_A(G_A, P):
    global  LAYER_A_TAG
    degree_c = sort_betweenness_centrality_desc(G_A)
    num = round(P*len(G_A.nodes()))
    return  [t[0] for t in list(degree_c.items())[:num]]

def attack_low_degree_Layer_A(G_A, P):
    global  LAYER_A_TAG
    degree_c = sort_degree_asc(G_A)
    num = round(P*len(G_A.nodes()))
    return  [t[0] for t in list(degree_c.items())[:num]]

def set_Layer_tag(G:nx.Graph, tag):
    G = nx.relabel_nodes(G, lambda i: tag+"_"+str(i))

    labels = [tag]
    global ATTR_LAYER
    nx.set_node_attributes(G, labels, ATTR_LAYER)
    return G
def draw_plot(x, measurement_ret, title):
    global FILE_TAG,FILE
    f = open(FILE,'a')
    print("## "+FILE_TAG, file=f)
    for idx in range(np.asarray(measurement_ret).shape[1]):
        new_fig()
        plt.title(title[idx])
        plt.plot(x, np.asarray(measurement_ret)[:, idx])
        print("#"+ title[idx], file=f)
        print("x="+str(x), file=f)
        print(FILE_TAG+title[idx]+"="+str(list(np.asarray(measurement_ret)[:, idx])), file=f)
    f.close()
    pass
##### experiment: Failure cascade
def experiment(layer_A_function, layer_B_function, interconnect_function, attack_strategy):
    global AVG_DEGREE,NUM_ITER
    num_iter = NUM_ITER
    ret_p = []
    x_p = []
    for p in np.arange(0.01, 1, 0.01):
        x_p.append(p)
        iter_ret=[]
        for i in range(num_iter):
            # Build interdependent networks
            ### construct Layer A
            G_A = layer_A_function(AVG_DEGREE)
            G_A = set_Layer_tag(G_A, LAYER_A_TAG)
            ### construct Layer B
            G_B = layer_B_function(AVG_DEGREE)
            G_B = set_Layer_tag(G_B, LAYER_B_TAG)

            ### interconnect
            G,layer_mapping = interconnect_function(G_A, G_B)

            # select nodes to attack
            attack_nodes = attack_strategy(G_A,(1-p))

            # simulate cascade
            iter_ret.append(np.asarray(simulate(G, G_A, G_B, layer_mapping, attack_nodes)))
        ret_p.append(np.mean(iter_ret, axis=0))
    draw_plot(x_p, ret_p, ['p_of_giant_mutual_connected_component_existing', 'largest_cluster_size_against_whole_networkk'])

layer_generate={"ER":lambda avg_d:generate_ER(avg_d), "BA":lambda avg_d:generate_BA(avg_d)}


interconnect_generate = {"Random":lambda G_A,G_B: interconnect_random(G_A, G_B),\
                         "Max_max_d": lambda  G_A,G_B:interconnect_max_max_degree_centralities(G_A,G_B),\
                         "Max_min_d": lambda G_A,G_B:interconnect_max_min_degree_centralities(G_A,G_B),\
                         "Min_max_d": lambda G_A,G_B:interconnect_min_max_degree_centralities(G_A,G_B),\
                         "Min_min_d": lambda G_A, G_B:interconnect_min_min_degree_centralities(G_A, G_B),
                         \
                         "Max_max_b": lambda  G_A,G_B:interconnect_max_max_betweeness(G_A,G_B),
                         "Max_min_b": lambda G_A,G_B:interconnect_max_min_betweeness(G_A,G_B),
                         "Min_max_b": lambda G_A,G_B:interconnect_min_max_betweeness(G_A,G_B),
                         "Min_min_b": lambda G_A, G_B:interconnect_min_min_betweeness(G_A, G_B),
                         }

attack_generate={"Random":lambda G,P: attack_random_Layer_A(G, P),\
                 "Max_d":lambda G,P: attack_high_degree_Layer_A(G, P),\
                 "Min_d": lambda G,P: attack_low_degree_Layer_A(G, P),\
                 "Max_b": lambda G,P: attack_high_betweenness_Layer_A(G, P)}

for idx in range(len(layer_a)):
    FILE_TAG = FILE_TAG + layer_a[idx] + "_" + layer_b[idx] +"_" + interconnect_stg[idx] + "_Con_"+ attack_stg[idx]+"_ATK"
    experiment(layer_generate[layer_a[idx]],layer_generate[layer_b[idx]],
           interconnect_generate[interconnect_stg[idx]],
           attack_generate[attack_stg[idx]])

